## Imports

In [74]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from ipaddress import *
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding, Dropout, Flatten, SpatialDropout1D, LSTM,Input, Concatenate, Reshape, InputLayer, concatenate
from tensorflow.keras.regularizers import l1, l2, l1_l2
import keras
import re
import tldextract
import warnings
warnings.filterwarnings("ignore")
sns.set_style(style='white') 

## Data Transforming

In [75]:

data=pd.read_csv('cleaned_data.csv')
df=data[['url','status']]
data=data.drop('url',axis=1)

In [76]:
data

,status,url_length,num_special_chars,num_digits,num_dots,num_slash,num_and,num_percent,domain_length,num_digits_in_domain,num_subdomains,subdomain_digits,is_HTTPS,depth,top_domain_encoded,domain_encoded,subdomain_encoded
0,0,39,0,7,2,0,0,0,12,0,1,7,0,1,225,111707,17
1,0,36,0,6,2,0,0,0,12,0,1,6,0,1,225,111707,18
2,0,12,0,5,1,0,0,0,5,5,0,0,0,1,878,10,0
3,0,35,0,4,2,0,0,0,12,0,1,4,0,1,225,111707,25
4,0,20,0,5,2,0,0,0,8,0,1,5,0,1,225,291146,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
798458,0,9,0,0,1,0,0,0,5,0,0,0,0,1,225,308751,0
798459,0,6,0,0,1,0,0,0,3,0,0,0,0,1,690,308750,0
798460,0,9,0,0,2,0,0,0,3,0,0,0,0,1,215,308771,0
798461,0,16,0,0,2,0,0,0,10,0,0,0,0,1,219,308776,0


In [77]:
a=data.top_domain_encoded.nunique()
data.top_domain_encoded.nunique()

1232

In [78]:
b=data.domain_encoded.nunique()
data.domain_encoded.nunique()

308783

In [79]:
c=data.subdomain_encoded.nunique()
data.subdomain_encoded.nunique()

111203

In [80]:
d=a+b+c
d

421218

In [81]:
data.head()

,status,url_length,num_special_chars,num_digits,num_dots,num_slash,num_and,num_percent,domain_length,num_digits_in_domain,num_subdomains,subdomain_digits,is_HTTPS,depth,top_domain_encoded,domain_encoded,subdomain_encoded
0,0,39,0,7,2,0,0,0,12,0,1,7,0,1,225,111707,17
1,0,36,0,6,2,0,0,0,12,0,1,6,0,1,225,111707,18
2,0,12,0,5,1,0,0,0,5,5,0,0,0,1,878,10,0
3,0,35,0,4,2,0,0,0,12,0,1,4,0,1,225,111707,25
4,0,20,0,5,2,0,0,0,8,0,1,5,0,1,225,291146,29


In [82]:

y=data['status']
X=data.drop('status', axis=1)
# col=X.columns
# X=StandardScaler().fit_transform(X)
# X=pd.DataFrame(X,columns=col)

In [83]:
X_train, X_test, y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=54)

In [84]:
X_train.shape

(638770, 16)

In [85]:
X_test.shape

(159693, 16)

In [86]:
X_train

,url_length,num_special_chars,num_digits,num_dots,num_slash,num_and,num_percent,domain_length,num_digits_in_domain,num_subdomains,subdomain_digits,is_HTTPS,depth,top_domain_encoded,domain_encoded,subdomain_encoded
411423,33,1,0,1,3,0,0,14,0,0,0,0,3,225,252330,0
80166,28,1,0,2,2,0,0,6,0,1,0,1,0,225,134982,4861
192551,61,0,0,3,4,0,0,13,0,0,0,0,5,225,276304,0
493917,59,0,4,2,5,0,0,4,0,0,0,0,6,495,94002,0
122843,43,0,0,4,3,0,0,6,0,2,0,0,0,1136,48034,5177
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224993,23,0,0,2,2,0,0,8,0,0,0,0,2,225,87508,0
267287,37,0,0,3,2,0,0,8,0,1,0,0,2,138,170447,46165
323343,41,0,0,2,2,0,0,8,0,1,0,0,3,225,158881,19396
367941,66,1,2,2,2,0,0,5,0,1,0,0,3,225,294355,46689


## LSTM

In [87]:


# # Input layers for categorical and numeric features
# input_top = Input(shape=(1,), name='top_domain_encoded')
# input_domain = Input(shape=(1,), name='domain_encoded')
# input_sub = Input(shape=(1,), name='subdomain_encoded')
# input_https = Input(shape=(1,), name='is_HTTPS')
# input_numeric = Input(shape=(1,12), name='numeric_input')


# embedding_dim = 12
# embedded_tld = Embedding(input_dim=a, output_dim=embedding_dim)(input_top)
# embedded_domain = Embedding(input_dim=b, output_dim=embedding_dim)(input_domain)
# embedded_sub = Embedding(input_dim=c, output_dim=embedding_dim)(input_sub)
# embedded_https = Embedding(input_dim=2, output_dim=embedding_dim)(input_https)

# embedded_tld = Reshape((1, embedding_dim))(embedded_tld)
# embedded_domain = Reshape((1, embedding_dim))(embedded_domain)
# embedded_sub = Reshape((1, embedding_dim))(embedded_sub)
# embedded_https = Reshape((1, embedding_dim))(embedded_https)


# numeric_reshaped = Reshape((1, 12))(input_numeric)
# concat_embeds = Concatenate(axis=1)([embedded_tld, embedded_domain, embedded_sub, embedded_https, numeric_reshaped])

# lstm_out = LSTM(32, activation='relu')(concat_embeds)
# dropout = Dropout(0.3)(lstm_out)
# output = Dense(1, activation='sigmoid', kernel_regularizer=l1(0.01), bias_regularizer=l2(0.01))(dropout)

# model = Model(inputs=[input_top, input_domain, input_sub, input_https, input_numeric], outputs=output)

# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy','precision', 'recall'])

# early_stop = EarlyStopping(monitor='val_loss', patience=3, verbose=1, restore_best_weights=True)

# numeric = X_train.drop(['top_domain_encoded', 'domain_encoded', 'subdomain_encoded', 'is_HTTPS'], axis=1)

# numeric_reshaped_train = np.expand_dims(numeric, axis=1)
# numeric_reshaped_test = np.expand_dims(X_test.drop(['top_domain_encoded', 'domain_encoded', 'subdomain_encoded', 'is_HTTPS'], axis=1), axis=1)

# # Fit the model
# num_epochs = 30
# historyL = model.fit([X_train['top_domain_encoded'], X_train['domain_encoded'], 
#                       X_train['subdomain_encoded'], X_train['is_HTTPS'], numeric_reshaped_train], 
#                       y_train, epochs=num_epochs, validation_data=([X_test['top_domain_encoded'], X_test['domain_encoded'], X_test['subdomain_encoded'], X_test['is_HTTPS'], numeric_reshaped_test], y_test), callbacks=[early_stop], verbose=2)


In [88]:
tf.keras.backend.clear_session()
tf.random.set_seed(50)
Lstm  = Sequential()
Lstm.add(Embedding(d,8))
Lstm.add(Dropout(0.2))
Lstm.add(LSTM(128,return_state=False))
Lstm.add(Dense(24,activation='relu'))
Lstm.add(Dense(1,activation ='sigmoid'))
# Lstm.add(Dense(1,activation ='sigmoid',kernel_regularizer=l1(0.01), bias_regularizer=l2(0.01)))

In [89]:
Lstm.compile(loss ='binary_crossentropy',optimizer='adam', metrics=['accuracy', 'precision', 'recall'])

In [90]:
num_epochs =30
early_stop = EarlyStopping(monitor='val_loss',patience=3, verbose=1, restore_best_weights=True)
historyL=Lstm.fit(X_train, y_train,epochs = num_epochs,validation_data=(X_test,y_test),callbacks =[early_stop],verbose =2)
# around8 minutes an epeoch when split 
# reached around88

Epoch 1/30


19962/19962 - 557s - 28ms/step - accuracy: 0.9303 - loss: 0.1640 - precision: 0.9255 - recall: 0.9458 - val_accuracy: 0.9597 - val_loss: 0.1003 - val_precision: 0.9608 - val_recall: 0.9638
Epoch 2/30
19962/19962 - 561s - 28ms/step - accuracy: 0.9822 - loss: 0.0506 - precision: 0.9834 - recall: 0.9832 - val_accuracy: 0.9519 - val_loss: 0.1183 - val_precision: 0.9559 - val_recall: 0.9540
Epoch 3/30
19962/19962 - 565s - 28ms/step - accuracy: 0.9902 - loss: 0.0297 - precision: 0.9906 - recall: 0.9910 - val_accuracy: 0.9535 - val_loss: 0.1219 - val_precision: 0.9551 - val_recall: 0.9580
Epoch 4/30
19962/19962 - 549s - 28ms/step - accuracy: 0.9929 - loss: 0.0220 - precision: 0.9930 - recall: 0.9938 - val_accuracy: 0.9528 - val_loss: 0.1250 - val_precision: 0.9541 - val_recall: 0.9577
Epoch 4: early stopping
Restoring model weights from the end of the best epoch: 1.
